In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import warnings
import numpy as np
import datetime
import math

In [2]:
df_wow_gsw = pd.read_csv('/Users/lauralayton/Downloads/output/_Grid_GSW2_2015.csv')
df_wow_tmin = pd.read_csv('/Users/lauralayton/Downloads/output/_Grid2_Daymet_V4_2015.csv')
df_wow_tmax = pd.read_csv('/Users/lauralayton/Downloads/output/_Grid2_Daymet_V4_tmax_2015.csv')
df_wow_ndvi = pd.read_csv('/Users/lauralayton/Downloads/WoW_NDVI_2015.csv')

In [3]:
df_wow_gsw

,CELL_AREA_M2,DATE,LAT,LONG,WETLAND_ID,water
0,659.516087,2015-01-01,42.740359,-81.246284,BL,0
1,659.516087,2015-01-01,42.740359,-81.246015,BL,0
2,659.513220,2015-01-01,42.740628,-81.246284,BL,0
3,659.513220,2015-01-01,42.740628,-81.246015,BL,0
4,659.510354,2015-01-01,42.740898,-81.246284,BL,0
...,...,...,...,...,...,...
1339,658.126962,2015-12-01,42.870794,-82.126454,OH,0
1340,658.126962,2015-12-01,42.870794,-82.126184,OH,0
1341,658.126962,2015-12-01,42.870794,-82.125915,OH,0
1342,658.124089,2015-12-01,42.871064,-82.126454,OH,0


In [4]:
df_wow_tmin

,CELL_AREA_M2,DATE,LAT,LONG,WETLAND_ID,tmin
0,659.516087,2015-01-01,42.740359,-81.246284,BL,-8.50
1,659.516087,2015-01-01,42.740359,-81.246015,BL,-8.50
2,659.513220,2015-01-01,42.740628,-81.246284,BL,-8.50
3,659.513220,2015-01-01,42.740628,-81.246015,BL,-8.50
4,659.510354,2015-01-01,42.740898,-81.246284,BL,-8.50
...,...,...,...,...,...,...
40875,658.126962,2015-12-31,42.870794,-82.126454,OH,-0.86
40876,658.126962,2015-12-31,42.870794,-82.126184,OH,-0.86
40877,658.126962,2015-12-31,42.870794,-82.125915,OH,-0.86
40878,658.124089,2015-12-31,42.871064,-82.126454,OH,-0.86


In [5]:
df_wow_tmax

,CELL_AREA_M2,DATE,LAT,LONG,WETLAND_ID,tmax
0,659.516087,2015-01-01,42.740359,-81.246284,BL,-1.71
1,659.516087,2015-01-01,42.740359,-81.246015,BL,-1.71
2,659.513220,2015-01-01,42.740628,-81.246284,BL,-1.71
3,659.513220,2015-01-01,42.740628,-81.246015,BL,-1.71
4,659.510354,2015-01-01,42.740898,-81.246284,BL,-1.71
...,...,...,...,...,...,...
40875,658.126962,2015-12-31,42.870794,-82.126454,OH,0.40
40876,658.126962,2015-12-31,42.870794,-82.126184,OH,0.40
40877,658.126962,2015-12-31,42.870794,-82.125915,OH,0.40
40878,658.124089,2015-12-31,42.871064,-82.126454,OH,0.40


In [6]:
df_wow_ndvi

,system:index,WETLAND_ID,imageId,mean,.geo
0,LC08_018030_20150110_00000000000000000000,BL,2015-01-10T16:03:50.870,-0.029215,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,LC08_018030_20150110_00000000000000000001,DY,2015-01-10T16:03:50.870,-0.015163,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,LC08_018030_20150110_00000000000000000002,FE,2015-01-10T16:03:50.870,-0.035827,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,LC08_018030_20150110_00000000000000000005,MA,2015-01-10T16:03:50.870,-0.027626,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,LC08_018030_20150126_00000000000000000000,BL,2015-01-26T16:03:45.004,-0.039711,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...
234,LC08_019030_20151203_00000000000000000004,LL,2015-12-03T16:10:07.180,0.047620,"{""type"":""MultiPoint"",""coordinates"":[]}"
235,LC08_019030_20151203_00000000000000000005,MA,2015-12-03T16:10:07.180,-0.034255,"{""type"":""MultiPoint"",""coordinates"":[]}"
236,LC08_019030_20151203_00000000000000000006,MO,2015-12-03T16:10:07.180,0.138974,"{""type"":""MultiPoint"",""coordinates"":[]}"
237,LC08_019030_20151203_00000000000000000007,OH,2015-12-03T16:10:07.180,0.194496,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [9]:
def water_ind(val):
    if val == 2:
        return 1
    else:
        return 0

def no_data_ind(val):
    if val == 0:
        return 1
    else:
        return 0

def indicators(df_gsw):
    df_gsw['NO_DATA_IND'] = df_gsw['water'].apply(no_data_ind)
    df_gsw['WATER_IND'] = df_gsw['water'].apply(water_ind)

indicators(df_wow_gsw)

df_wow_gsw['HYDROPERIOD'] = np.nan
df_wow_gsw['MONTHS_NO_DATA'] = np.nan
hloc = df_wow_gsw.columns.get_loc('HYDROPERIOD')
mloc = df_wow_gsw.columns.get_loc('MONTHS_NO_DATA')

for index, row in df_wow_gsw.iterrows():
    lat = row['LAT']
    long = row['LONG']
    wID = row['WETLAND_ID']
    
    df_subset = df_wow_gsw[(df_wow_gsw['LAT'] == lat) & 
        (df_wow_gsw['LONG'] == long) & 
        (df_wow_gsw['WETLAND_ID'] == wID)]

    hydroperiod = 30.5 * sum(df_subset['WATER_IND'])
    mon_no_data = sum(df_subset['NO_DATA_IND'])

    df_wow_gsw.iloc[index, hloc] = hydroperiod
    df_wow_gsw.iloc[index, mloc] = mon_no_data

In [12]:
df_wow_gsw[['HYDROPERIOD', 'WETLAND_ID']].value_counts()

HYDROPERIOD  WETLAND_ID
0.0          OH            252
             LL            240
             FE            204
             KE            156
             MA            132
             DY            120
             BL            108
             MO            108
122.0        FE             24
Name: count, dtype: int64